# Exercício de Integração ETL, ELT, APIs e Arquivos


## Trabalho em Grupo

Você trabalha em um e-commerce de variados produtos, e no último mês os gerentes operacionais da companhia notaram que
houve uma variação muito grande em seus estoques por períodos de até 24h, cujos produtos, voltavam ao estoque
no dia seguinte o que não caracterizava uma venda concretizada.
Sabendo disso um dos diretores sugeriu um estudo sobre os carrinhos em aberto, e qual o perfil dos usuários que deixavam seus carrinhos em aberto, e para isso fora criada uma demanda para seu time.

Para resulver esta demanda, você e seu time deverão realizar a ingestão dos dados de:

- Usuários
- Carrinhos

Estes dados poderão ser obtidos através dos seguintes endpoints:

- Usuários: https://dummyjson.com/auth/users
- Carrinhos: https://dummyjson.com/auth/carts

Lembrando que estes são endpoints seguros então você deverá realizar a autenticação através do seguinte endpoint: https://dummyjson.com/auth/login

Lembre-se de verificar a documentação para obter mais detalhes do funcionamento dos endpoints https://dummyjson.com/docs/auth

Após isso, os passos de tratamento do dado, serão:

1. Após consumidos os dados, você deverá persistí-los utilizando a técnica aprendida em aula em uma pastas chamada raw no formato json

2. Consumir sua camada raw e criar uma nova entidade que relacione os clientes aos carrihos através do atributo userId do seu carrinho e o atribute id da tabela users, feito isso gravar o resultado dessa agregação em uma pasta chamda kitchen

3. Por fim você deverá contemplar se os dados gerados são capazes de responder as seguintes perguntas:

   - Qual o estado de residencia possui clientes com mais carrinhos em aberto ?
   - Qual a idade dos usuários com mais carrinhos em aberto ?
   - Quais os produtos que mais aparecem em carrinhos em aberto ?

OBS: **Não se preocupe em fazer uma análise aprofundade, o intuito desta atividade é exercitar a construção de um fluxo de ETL ou ELT utilizando os conceitos aprendidos em sala de aula, então concentrem seus esforços no fluxo de dados**


In [ ]:
import pandas as pd
import requests
import json
from io import StringIO


def authenticate(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "username": login,
            "password": senha,
            "expiresInMins": 1
        })
    )

    return login_response.json()['token']


def get_authorization_headers():
    access_token = authenticate(login="kminchelle", senha="0lelplR")

    authorization_header = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    return authorization_header


def get_endpoint(url: str):
    response = requests.get(
        url=url,
        headers=get_authorization_headers()
    )

    return response.json()

In [ ]:
users = get_endpoint("https://dummyjson.com/auth/users")
carts = get_endpoint("https://dummyjson.com/auth/carts")

json_users = json.dumps(users)
json_carts = json.dumps(carts)


def file_convert_text(type: str, data: str, path: str):
    with open(f"../ex_full_cycle/{path}/{type}.json", mode="w") as file_to_write:
        file_to_write.write(data)
        file_to_write.flush()
        file_to_write.close()


file_convert_text("users", json_users, "raw_full_cycle")
file_convert_text("carts", json_carts, "raw_full_cycle")

In [127]:
def file_open(type: str):
    users_created = pd.read_json(
        f"../ex_full_cycle/raw_full_cycle/{type}.json", encoding="utf-8")
    return users_created


get_users = file_open("users")
get_carts = file_open("carts")

expanded_users = pd.DataFrame(pd.json_normalize(get_users['users']))
expanded_carts = pd.DataFrame(pd.json_normalize(get_carts['carts']))

In [131]:
# Operações de normalização
expanded_users.rename(columns={"id": "userId"}, inplace=True)

join_merged = pd.merge(expanded_users, expanded_carts,
                       on="userId", how="inner")

json_string = join_merged.to_json(orient='records')

# Salvando json na pasta 'kitchen'
file_convert_text("join_merged", json_string, "kitchen")


# Expadindo produtos e resetando index
group_products = pd.DataFrame(join_merged['products']).explode(
    'products').apply(pd.Series).reset_index(drop=True)["products"]

#
df_from_json = pd.read_json(StringIO(group_products.to_json(orient='records')))

print(type(df_from_json))


group_products = df_from_json.groupby("title").size().reset_index(
    name='count').sort_values(by="count", ascending=False)

group_products.rename(
    columns={"count":  "count_products_view", "title": "name_products"}, inplace=True)

adress_and_age = pd.DataFrame(
    join_merged[['userId', 'firstName', 'address.state', 'age']])

adress_and_age.rename(
    columns={"age": "age_user", "address.state": "address_state", "firstName": "first_name", "userId": "user_id"}, inplace=True)

display(group_products)
display(adress_and_age)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


,name_products,count_products_view
23,lighting ceiling kitchen,2
18,Women Shoes,2
16,Seven Pocket Women Bag,2
15,Samsung Universe 9,2
0,- Daal Masoor 500 grams,1
1,Automatic Motor Gas Motorcycles,1
24,perfume Oil,1
22,iphone X,1
21,frock gold printed,1
20,ank Tops for Womens/Girls,1


,user_id,first_name,address_state,age_user
0,1,Terry,DC,50
1,5,Mavis,KY,38
2,13,Trace,VT,26
3,15,Jeanne,MA,26
4,26,Griffin,AK,35
5,30,Maurine,DC,31
